In [1]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib

import numpy as np
import pandas as pd

## Panel:
import param
import panel as pn
from panel.viewable import Viewer

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import PrettyPrintable, get_arguments_as_optional_dict
from pyphocorehelpers.function_helpers import compose_functions

from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import * # get_neuron_identities
from pyphoplacecellanalysis.General.SessionSelectionAndFiltering import batch_filter_session
from pyphoplacecellanalysis.General.ComputationResults import ComputationResult
# from PendingNotebookCode import estimation_session_laps


from neuropy.analyses.laps import estimation_session_laps
from neuropy.core.epoch import NamedTimerange

# Neuropy:
from neuropy.analyses.placefields import PlacefieldComputationParameters, perform_compute_placefields
from neuropy.core.neuron_identities import NeuronIdentity, build_units_colormap, PlotStringBrevityModeEnum
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_spike_counts, debug_print_subsession_neuron_differences

# def estimate_session_laps_load_function(regular_load_function, a_base_dir):
#     session = regular_load_function(a_base_dir)
#     ## Estimate the Session's Laps data using my algorithm from the loaded position data.
#     session = estimation_session_laps(session)
#     return session

known_data_session_type_dict = {'kdiba':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.kdiba_old_format_session(a_base_dir)),
                               basedir=Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')),
                'bapun':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.bapun_data_session(a_base_dir)),
                               basedir=Path('R:\data\Bapun\Day5TwoNovel'))
               }

known_data_session_type_dict['kdiba'].post_load_functions = [lambda a_loaded_sess: estimation_session_laps(a_loaded_sess)]
# known_data_session_type_dict = {'kdiba':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: estimation_session_laps(DataSessionLoader.kdiba_old_format_session(a_base_dir))),
#                                basedir=Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')),
#                 'bapun':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.bapun_data_session(a_base_dir)),
#                                basedir=Path('R:\data\Bapun\Day5TwoNovel'))
#                }

# known_data_session_type_dict['kdiba'].name

neuropy module not found, adding directory to sys.path. 
 >> Updated sys.path.


In [2]:
# Truely Common:
def _display_result(computation_result):
    pf_neuron_identities, pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = get_neuron_identities(computation_result.computed_data['pf2D'])
    computation_result.computed_data['pf2D'].plot_raw(label_cells=True); # Plots an overview of each cell all in one figure
    computation_result.computed_data['pf2D'].plot_ratemaps_2D(resolution_multiplier=2.5, brev_mode=PlotStringBrevityModeEnum.MINIMAL)

    

# Bapun Format:

In [ ]:
# curr_bapun_pipeline = NeuropyPipeline(name='bapun_pipeline', session_data_type='bapun', basedir=known_data_session_type_dict['bapun'].basedir, load_function=known_data_session_type_dict['bapun'].load_function)
curr_bapun_pipeline = NeuropyPipeline.init_from_known_data_session_type('bapun', known_data_session_type_dict['bapun'])

# curr_bapun_pipeline = NeuropyPipeline.init_from_known_data_session_type('bapun', known_data_session_type_dict['bapun'])
curr_bapun_pipeline.is_loaded
size_bytes = curr_bapun_pipeline.sess.__sizeof__() # 1753723032
f'object size: {size_bytes/(1024*1024)} MB'

In [ ]:
# Bapun/DataFrame style session filter functions:
def _temp_filter_session_by_epoch1(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze1')
    ## All Spikes:
    # active_epoch_session = sess.filtered_by_epoch(active_epoch) # old
    active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch

def _temp_filter_session_by_epoch2(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze2')
    ## All Spikes:
    # active_epoch_session = sess.filtered_by_epoch(active_epoch) # old
    active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch


active_session_filter_configurations = {'maze1':_temp_filter_session_by_epoch1,
                                        'maze2':_temp_filter_session_by_epoch2
                                       }

curr_bapun_pipeline.filter_sessions(active_session_filter_configurations)
curr_bapun_pipeline.perform_computations(PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1))

In [ ]:
_display_result(curr_bapun_pipeline.computation_results['maze1'])

In [ ]:
_display_result(curr_bapun_pipeline.computation_results['maze2'])

# KDiba Format:

In [13]:
## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m
# From pre-computed .mat files:
## 07: 
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
# # ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
# curr_kdiba_pipeline = NeuropyPipeline(name='kdiba_pipeline', session_data_type='kdiba', basedir=known_data_session_type_dict['kdiba'].basedir, load_function=known_data_session_type_dict['kdiba'].load_function)
curr_kdiba_pipeline = NeuropyPipeline.init_from_known_data_session_type('kdiba', known_data_session_type_dict['kdiba'])

# curr_bapun_pipeline
curr_kdiba_pipeline.is_loaded
size_bytes = curr_kdiba_pipeline.sess.__sizeof__() # 1753723032
f'object size: {size_bytes/(1024*1024)} MB'
# ## Estimate the Session's Laps data using my algorithm from the loaded position data.
# curr_kdiba_pipeline.sess = estimation_session_laps(curr_kdiba_pipeline.sess)
curr_kdiba_pipeline.sess.epochs

C:\Users\Pho\repos\NeuroPy\neuropy\core\position.py:23: UserWarning: registration of accessor <class 'neuropy.core.position.PositionAccessor'> under name 'position' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PositionAccessor(TimeSlicedMixin):


basedir is already Path object.
	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy... 2006-6-07_11-26-53.position.npy saved
done.
	 Failure loading .interpolated_spike_positions.npy. Must recompute.

	 Saving updated interpolated spike position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.interpolated_spike_positions.npy.

         start         stop  label     duration
0     0.000000  1739.153364  maze1  1739.153364
1  1739.153364  1932.420005  maze2   193.266641

In [14]:
def _temp_filter_session_by_epoch1(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze1')
    ## All Spikes:
    active_session = sess.filtered_by_epoch(active_epoch) # kdiba
    # active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch

def _temp_filter_session_by_epoch2(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze2')
    ## All Spikes:
    active_session = sess.filtered_by_epoch(active_epoch) # kdiba
    # active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch()) # new Bapun/Df
    return active_session, active_epoch

active_session_filter_configurations = {'maze1':_temp_filter_session_by_epoch1,
                                        'maze2':_temp_filter_session_by_epoch2
                                       }

# active_config = build_configs(sess.config, active_epoch)

curr_kdiba_pipeline.filter_sessions(active_session_filter_configurations)
curr_kdiba_pipeline.perform_computations(PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1))

Applying session filter named "maze1"...
Constraining to epoch with times (start: 0.0, end: 1739.1533641185379)
Applying session filter named "maze2"...
Constraining to epoch with times (start: 1739.1533641185379, end: 1932.4200048116618)
Performing single_computation on filtered_session with filter named "maze1"...
Recomputing active_epoch_placefields... 	 done.
Recomputing active_epoch_placefields2D... 	 done.
Performing single_computation on filtered_session with filter named "maze2"...
Recomputing active_epoch_placefields... 	 done.
Recomputing active_epoch_placefields2D... 	 done.


In [15]:
def build_lap_epochs_filters(sess):
    lap_specific_epochs = sess.laps.as_epoch_obj()
    # lap_specific_epochs.to_dataframe()
    any_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(len(sess.laps.lap_id))])
    even_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(0, len(sess.laps.lap_id), 2)])
    odd_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(1, len(sess.laps.lap_id), 2)])
    
    ## All Spikes:
    sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
    # sess.epochs.to_dataframe()
    # active_epoch = sess.epochs.get_named_timerange('maze1')
    # print('active_epoch: {}'.format(active_epoch))
    # active_epoch = sess.epochs.get_named_timerange('maze2')
    # active_epoch_maze_all = NamedTimerange(name='maze', start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]])

    # active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
    # print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)

    active_session_filter_configurations = {'maze1': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze1')), x.epochs.get_named_timerange('maze1')),
                                        'maze2': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze2')), x.epochs.get_named_timerange('maze2')),
                                        'maze': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]])), NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]]))
                                       }
    return active_session_filter_configurations
    
active_session_filter_configurations = build_lap_epochs_filters(curr_kdiba_pipeline.sess)
curr_kdiba_pipeline.filter_sessions(active_session_filter_configurations)

Applying session filter named "maze1"...
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 22.26, end: 1739.1533641185379)
Applying session filter named "maze2"...
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 1739.1533641185379, end: 1932.4200048116618)
Applying session filter named "maze"...
Constraining to units with type: pyramidal
Constraining to epoch with times (start: 22.26, end: 1932.4200048116618)


In [ ]:
def debug_plot_filtered_subsession_neuron_differences(sess, filtered_sess):
    print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)

[debug_plot_filtered_subsession_neuron_differences(curr_kdiba_pipeline.sess, a_filtered_sess) for a_filtered_sess in curr_kdiba_pipeline.filtered_sessions]

In [ ]:
curr_kdiba_pipeline.computation_results['maze2']

In [ ]:
_display_result(curr_kdiba_pipeline.computation_results['maze1'])
_display_result(curr_kdiba_pipeline.computation_results['maze2'])

In [ ]:
def _display_normal(computation_result, active_config):
    from neuropy.plotting.placemaps import plot_all_placefields
    # from neuropy.core.neuron_identities import build_units_colormap
    # print(f'active_config: {active_config}')
    # active_config = computation_result.sess.config
    if active_config.computation_config is None:
        active_config.computation_config = computation_result.computation_config

    pf_neuron_identities, pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = get_neuron_identities(computation_result.computed_data['pf2D'])
    ax_pf_1D, occupancy_fig, active_pf_2D_figures, active_pf_2D_gs = plot_all_placefields(computation_result.computed_data['pf1D'], computation_result.computed_data['pf2D'], active_config, should_save_to_disk=False)
    
# curr_kdiba_pipeline.computation_results['maze1'].computation_config
# curr_kdiba_pipeline.computation_results['maze1'].sess.config
# curr_kdiba_pipeline.active_configs['maze1']
_display_normal(curr_kdiba_pipeline.computation_results['maze1'], curr_kdiba_pipeline.active_configs['maze1'])

In [ ]:
def _display_testing(sess, computation_result, active_config):
    """ Testing of plot_lap_trajectories_2d """
    # from PhoPositionalData.plotting.laps import plot_lap_trajectories_2d
    # fig, axs, laps_pages = plot_lap_trajectories_2d(sess, curr_num_subplots=len(sess.laps.lap_id), active_page_index=0)
    # fig.suptitle('Lap Trajectories 2D', fontsize=22)
    # # fig_out_path = active_config.plotting_config.get_figure_save_path('lap_trajectories_2D').with_suffix('.png')
    # # fig.savefig(fig_out_path)
    # # plt.show()

    from PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
    p, laps_pages = plot_lap_trajectories_3d(sess, curr_num_subplots=10, active_page_index=1)
    return p


    
# curr_kdiba_pipeline.computation_results['maze1'].computation_config
# curr_kdiba_pipeline.computation_results['maze1'].sess.config
# curr_kdiba_pipeline.active_configs['maze1']
p = _display_testing(curr_kdiba_pipeline.sess, curr_kdiba_pipeline.computation_results['maze1'], curr_kdiba_pipeline.active_configs['maze1'])
p.show()






In [43]:
from itertools import islice # for Pagination class
import pyvista as pv
import pyvistaqt as pvqt
from PhoGui.InteractivePlotter.LapsVisualizationMixin import LapsVisualizationMixin
from PhoGui.PhoCustomVtkWidgets import PhoWidgetHelper
from PhoPositionalData.plotting.spikeAndPositions import perform_plot_flat_arena

In [41]:
""" Test Drawing Spike Lines """
from pyphoplacecellanalysis.Pho3D.spikes import draw_line_spike
# draw_line_spike(p, 

get_arguments_as_optional_dict(name='maze_bg', label='maze', scalars='traversal_order_scalars', render=True)

, **({'name': 'maze_bg', 'label': 'maze', 'scalars': 'traversal_order_scalars', 'render': True} | kwargs)


In [48]:
def _plot_lap_trajectories_combined_plot_3d(sess, curr_num_subplots=1, active_page_index=0, single_combined_plot=True):
    """ Plots a PyVista Qt Multiplotter with several overhead 3D views, each showing a specific lap over the maze in one of its subplots 
    Usage: 
        p, laps_pages = plot_lap_trajectories_3d(sess, curr_num_subplots=10, active_page_index=1)
        p.show()
    """
    def _chunks(iterable, size=10):
        iterator = iter(iterable)
        for first in iterator:    # stops when iterator is depleted
            def chunk():          # construct generator for next chunk
                yield first       # yield element from for loop
                for more in islice(iterator, size - 1):
                    yield more    # yield more elements from the iterator
            yield chunk()         # in outer generator, yield next chunk

    def _compute_laps_position_data(sess):
        curr_position_df = sess.compute_position_laps()
        lap_specific_position_dfs = [curr_position_df.groupby('lap').get_group(i)[['t','x','y','lin_pos']] for i in sess.laps.lap_id] # dataframes split for each ID:
        return curr_position_df, lap_specific_position_dfs
        
    def _build_laps_multiplotter(nfields, linear_plot_data=None, maximum_fixed_columns:int=5):
        linear_plotter_indicies = np.arange(nfields)
        fixed_columns = min(maximum_fixed_columns, nfields)
        needed_rows = int(np.ceil(nfields / fixed_columns))
        row_column_indicies = np.unravel_index(linear_plotter_indicies, (needed_rows, fixed_columns)) # inverse is: np.ravel_multi_index(row_column_indicies, (needed_rows, fixed_columns))
        mp = pvqt.MultiPlotter(nrows=needed_rows, ncols=fixed_columns, show=False, title='Laps Muliplotter', toolbar=False, menu_bar=False, editor=False)
        # print('linear_plotter_indicies: {}\n row_column_indicies: {}\n'.format(linear_plotter_indicies, row_column_indicies))
        for a_linear_index in linear_plotter_indicies:
            # print('a_linear_index: {}, row_column_indicies[0][a_linear_index]: {}, row_column_indicies[1][a_linear_index]: {}'.format(a_linear_index, row_column_indicies[0][a_linear_index], row_column_indicies[1][a_linear_index]))
            curr_row = row_column_indicies[0][a_linear_index]
            curr_col = row_column_indicies[1][a_linear_index]
            if linear_plot_data is None:
                mp[curr_row, curr_col].add_mesh(pv.Sphere())
            else:
                # mp[curr_row, curr_col].add_mesh(linear_plot_data[a_linear_index], name='maze_bg', color="black", render=False)
                perform_plot_flat_arena(mp[curr_row, curr_col], linear_plot_data[0], linear_plot_data[1], z=-0.01, name='maze_bg', render=False)
        return mp, linear_plotter_indicies, row_column_indicies

    
    def _add_specific_lap_trajectory(p, linear_plotter_indicies, row_column_indicies, active_page_laps_ids, curr_lap_position_traces, curr_lap_time_range, single_combined_plot: bool):
        # Add the lap trajectory:
        for a_linear_index in linear_plotter_indicies:
            curr_row = row_column_indicies[0][a_linear_index]
            curr_col = row_column_indicies[1][a_linear_index]
            if single_combined_plot:
                # curr_lap_id = active_page_laps_ids[a_linear_index]
                # print(f'curr_lap_id: {curr_lap_id}')
                for curr_lap_idx, curr_lap_id in enumerate(active_page_laps_ids):
                    LapsVisualizationMixin.plot_lap_trajectory_path_spline(p[curr_row, curr_col], curr_lap_position_traces[curr_lap_idx], curr_lap_id, lap_id_dependent_z_offset=1.0)
                    # curr_lap_label_text = 'Lap[{}]: t({:.2f}, {:.2f})'.format(curr_lap_id, curr_lap_time_range[curr_lap_id][0], curr_lap_time_range[curr_lap_id][1]) 
                    # PhoWidgetHelper.perform_add_text(p[curr_row, curr_col], curr_lap_label_text, name='lblLapIdIndicator')
            else:
                curr_lap_id = active_page_laps_ids[a_linear_index]
                LapsVisualizationMixin.plot_lap_trajectory_path_spline(p[curr_row, curr_col], curr_lap_position_traces[curr_lap_id], a_linear_index)
                curr_lap_label_text = 'Lap[{}]: t({:.2f}, {:.2f})'.format(curr_lap_id, curr_lap_time_range[curr_lap_id][0], curr_lap_time_range[curr_lap_id][1]) 
                PhoWidgetHelper.perform_add_text(p[curr_row, curr_col], curr_lap_label_text, name='lblLapIdIndicator')

    # Compute required data from session:
    curr_position_df, lap_specific_position_dfs = _compute_laps_position_data(sess)
    curr_lap_position_traces = [lap_pos_df[['x','y']].to_numpy().T for lap_pos_df in lap_specific_position_dfs]
    curr_lap_time_range = [[lap_pos_df[['t']].to_numpy()[0].item(), lap_pos_df[['t']].to_numpy()[-1].item()] for lap_pos_df in lap_specific_position_dfs]

    all_maze_positions = curr_position_df[['x','y']].to_numpy().T # (2, 59308)
    # np.shape(all_maze_positions)
    
    # perform_plot_flat_arena(all_maze_positions[0,:], all_maze_positions[1,:], z=-0.01, smoothing=True)
    
    

    if single_combined_plot:
        all_maze_data = (all_maze_positions[0,:], all_maze_positions[1,:])
    else:
        pdata_maze_shared, pc_maze_shared = __build_flat_map_plot_data(all_maze_positions[0,:], all_maze_positions[1,:])
        all_maze_data = np.full((curr_num_subplots,), pc_maze_shared) # repeat the maze data for each subplot
        
    p, linear_plotter_indicies, row_column_indicies = _build_laps_multiplotter(curr_num_subplots, all_maze_data)
    # generate the pages
    if single_combined_plot:
        laps_pages = [list(sess.laps.lap_id)] # single 'page'
    else:
        laps_pages = [list(chunk) for chunk in _chunks(sess.laps.lap_id, curr_num_subplots)]
    active_page_laps_ids = laps_pages[active_page_index]
    # print(f'active_page_laps_ids: {active_page_laps_ids}, curr_lap_position_traces: {curr_lap_position_traces}')
    if single_combined_plot:
        lap_id_dependent_z_offset = 1.0
        for curr_lap_idx, curr_lap_id in enumerate(active_page_laps_ids):
            curr_maze_z_offset = -0.01 + (lap_id_dependent_z_offset * (curr_lap_idx + 1))
            perform_plot_flat_arena(p[0,0], all_maze_data[0], all_maze_data[1], z=curr_maze_z_offset, name=f'maze_offset[{curr_lap_idx}]', render=False)

    # add the laps
    _add_specific_lap_trajectory(p, linear_plotter_indicies, row_column_indicies, active_page_laps_ids, curr_lap_position_traces, curr_lap_time_range, single_combined_plot=single_combined_plot)
    return p, laps_pages





p, laps_pages = _plot_lap_trajectories_combined_plot_3d(curr_kdiba_pipeline.sess, curr_num_subplots=1)
p.show()
laps_pages

NameError: name 'curr_row' is not defined